In [2]:
# Importar bibliotecas
import numpy as np
import pandas as pd
import geopandas as gpd


# Introdução

## Introdução ao Problema
O objetivo deste algoritmo é gerar um conjunto de pares de endereços na cidade de São Paulo para o treinamento de um modelo de precificação de corridas de Uber.

O algoritmo terá natureza estocástica, dando preferência para endereços de chegada em distritos com maior densidade populacional e renda média por habitante (fatores relevantes para o uso do aplicativo Uber).

Para os endereços de saída, o algoritmo usará uma função de decaimento exponencial $P(d)$ (descrita abaixo) para gerar uma distância máxima para o endereço do destino, onde escolherá o endereço de destino uniformemente dentro desse intervalo.

Usaremos o conjunto de dados geográficos (disponível como um arquivo `.shape` com um grafo representando as ruas de São Paulo) distribuído pela plataforma [GeoSampa](https://geosampa.prefeitura.sp.gov.br) da Prefeitura de São Paulo para acessar os endereços disponíveis dentro da distância $d$ da origem.

### Implementação em Código Python

A implementação segue as etapas principais descritas abaixo:

1. __Carregamento e Preparação dos Dados:__

   - Utilize bibliotecas como `geopandas` para manipular arquivos `.shapefile`.
   - Construa o grafo das ruas de São Paulo com `networkx`.
2. __Cálculo das Probabilidades de Escolha:__

   - Importe os dados do IBGE utilizando `pandas`.
   - Calcule e normalize as probabilidades com base na população e renda dos distritos.
3. __Seleção do Endereço de Origem:__

   - Escolha aleatória de uma rua com `numpy.random.choice`.
   - Geração de um número dentro do intervalo válido.
4. __Seleção do Endereço de Destino:__

   - Use $P(d)$ para calcular o raio máximo:

     $$
     P(d) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{d^2}{2\sigma^2}}
     $$
   - Escolha uniformemente dentro do raio usando coordenadas do grafo.
5. **Visualização dos Resultados:**

   - Use `matplotlib` ou `folium` para criar mapas interativos.





### Visualização e Validação

Após a implementação, as corridas simuladas podem ser visualizadas em mapas interativos:

- **Mapas com `folium`:** Representam pares de origem e destino.
- **Gráficos:** Analisam estatisticamente a distribuição das distâncias e destinos.

Além disso, as corridas geradas podem ser exportadas para arquivos `.csv` para uso em modelos de treinamento.

## Preparação dos Dados

# Escolha do Endereço de Chegada

## Cálculo Das Probabilidades de Escolha de Cada Distrito

Queremos selecionar os distritos oficiais de São Paulo e calcular a probabilidade de escolha de cada um deles. Utilizamos a base de dados do IBGE, que contém a população e a renda média de cada distrito. O cálculo da probabilidade de escolha segue a fórmula:

$$
\text{Probabilidade(distrito)} = \frac{\text{População(distrito)} \cdot \text{RendaMédia(distrito)}}{\sum_{distritos} (\text{População} \cdot \text{RendaMédia})}
$$

Essa probabilidade será utilizada para realizar a amostragem ponderada entre os distritos.

In [3]:
# Carregar os dados
df = pd.read_csv('data/population_parameters_rates_by_district.csv')
df = df[['Cod_distrito', 'Nome_distrito', 'Pop_2010', 'Pop_2020', 'Pop/ha_2010']]
df_renda = pd.read_csv('data/renda_media_distrito.csv')

# Remover espaços em branco e converter para minúsculas
df['Nome_distrito'] = df['Nome_distrito'].str.strip().str.lower()
df_renda['Nome_distrito'] = df_renda['Nome_distrito'].str.strip().str.lower()

# Verificar valores ausentes
print(df['Nome_distrito'].isnull().sum())
print(df_renda['Nome_distrito'].isnull().sum())

# Definir intervalos de renda e seus pontos médios
income_intervals = {
    'Menos de 2 SM': 1,
    'De 2 a Menos de 5 SM': 3.5,
    'De 5 a Menos de 10 SM': 7.5,
    'De 10 a Menos de 15 SM': 12.5,
    'De 15 a Menos de 25 SM': 20,
    'De 25 SM e Mais ': 30
}

# Calcular a renda média para cada distrito
for col, midpoint in income_intervals.items():
    df_renda[col] = df_renda[col]/100 * midpoint

df_renda['average_income'] = df_renda[list(income_intervals.keys())].sum(axis=1)

# Normalizar a renda média
min_income = df_renda['average_income'].min()
max_income = df_renda['average_income'].max()
df_renda['normalized_income'] = (df_renda['average_income'] - min_income) / (max_income - min_income)

# Normalizar a população
min_pop = df['Pop_2020'].min()
max_pop = df['Pop_2020'].max()
df['normalized_pop'] = (df['Pop_2020'] - min_pop) / (max_pop - min_pop)

# Mesclar os dois dataframes na coluna 'Nome_distrito'
result = pd.merge(df, df_renda, on='Nome_distrito')

# Calcular a pontuação composta (média simples das normalizações)
result['composite_score'] = (result['normalized_income'] + result['normalized_pop']) / 2

# Normalizar a pontuação composta para obter probabilidades
total_score = result['composite_score'].sum()
result['probability'] = result['composite_score'] / total_score

# Exibir o resultado final com as probabilidades
print(result[['Nome_distrito', 'average_income', 'Pop_2020', 'normalized_income', 'normalized_pop', 'composite_score', 'probability']].head())

0
0
       Nome_distrito  average_income  Pop_2020  normalized_income  \
0          agua rasa         12.9614     80618           0.474289   
1  alto de pinheiros         21.5901     40117           0.920206   
2         anhanguera          6.3032    108277           0.130204   
3         aricanduva          9.8699     81265           0.314525   
4        artur alvim          9.2310     95587           0.281508   

   normalized_pop  composite_score  probability  
0        0.198620         0.336454     0.009768  
1        0.088173         0.504189     0.014637  
2        0.274047         0.202125     0.005868  
3        0.200385         0.257455     0.007474  
4        0.239441         0.260474     0.007562  


In [4]:
prob_distrito_de_origem = result[['Nome_distrito','Pop/ha_2010', 'average_income']]
prob_distrito_de_origem['Pop/ha_2010_normalized'] = prob_distrito_de_origem['Pop/ha_2010'] / prob_distrito_de_origem['Pop/ha_2010'].sum()
prob_distrito_de_origem['average_income_normalized'] = prob_distrito_de_origem['average_income'] / prob_distrito_de_origem['average_income'].sum()
prob_distrito_de_origem['probability'] = (prob_distrito_de_origem['Pop/ha_2010_normalized'] + prob_distrito_de_origem['average_income_normalized']) / 2


/tmp/ipykernel_13697/211545537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_distrito_de_origem['Pop/ha_2010_normalized'] = prob_distrito_de_origem['Pop/ha_2010'] / prob_distrito_de_origem['Pop/ha_2010'].sum()
/tmp/ipykernel_13697/211545537.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_distrito_de_origem['average_income_normalized'] = prob_distrito_de_origem['average_income'] / prob_distrito_de_origem['average_income'].sum()
/tmp/ipykernel_13697/211545537.py:4: SettingWithCopyWarning: 

In [5]:
prob_distrito_de_origem = prob_distrito_de_origem[['Nome_distrito', 'probability']] # Retirando colunas auxiliares do cálculo da probabilidade
prob_distrito_de_origem

,Nome_distrito,probability
0,agua rasa,0.011681
1,alto de pinheiros,0.012339
2,anhanguera,0.003766
3,aricanduva,0.010901
4,artur alvim,0.011747
...,...,...
90,vila medeiros,0.012249
91,vila prudente,0.010210
92,vila sonia,0.011983
93,sao domingos,0.009060


In [7]:
# 1. Introdução
# Inicialização do Geo-dataframe da cidade de São Paulo


gdf = gpd.read_file('data/mapas/SAD69-96_SHP/SAD69-96_SHP_logradouronbl_line.shp')
gdf = gdf[gdf['lg_tipo'].isin(['R', 'AV'])] # Selecionar apenas ruas e avenidas
# gdf = gdf[['lg_nome', '']]


In [8]:
def selecionar_endereço_de_origem(prob_distrito_de_origem, geodataframe=gdf):
    distritos = prob_distrito_de_origem['Nome_distrito']
    probabilidades = prob_distrito_de_origem['probability']
    distrito_amostrado = np.random.choice(distritos, p=probabilidades)
    print(f"Distrito amostrado: {distrito_amostrado}")
    df_ruas = geodataframe[geodataframe['distrito'] == distrito_amostrado]

In [12]:
gpd.read_file('data/ibge/SP_Municipios_2023.shp').head

<bound method NDFrame.head of       CD_MUN            NM_MUN  CD_RGI                 NM_RGI CD_RGINT  \
0    3500105        Adamantina  350019   Adamantina - Lucélia     3505   
1    3500204            Adolfo  350025  São José do Rio Preto     3507   
2    3500303             Aguaí  350044  São João da Boa Vista     3510   
3    3500402    Águas da Prata  350044  São João da Boa Vista     3510   
4    3500501  Águas de Lindóia  350048                 Amparo     3510   
..       ...               ...     ...                    ...      ...   
640  3557006        Votorantim  350003               Sorocaba     3502   
641  3557105       Votuporanga  350027            Votuporanga     3507   
642  3557154          Zacarias  350023    Birigui - Penápolis     3506   
643  3557204         Chavantes  350015               Ourinhos     3504   
644  3557303      Estiva Gerbi  350043             Mogi Guaçu     3510   

                  NM_RGINT CD_UF      NM_UF CD_REGIAO NM_REGIAO CD_CONCURB  \
0  